In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize, poly, sklearn_sm, Stepwise, sklearn_selected, sklearn_selection_path)
from functools import partial
import statsmodels.api as sm

In [38]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import KFold, cross_validate, cross_val_predict, GridSearchCV
from sklearn.linear_model import Ridge, Lasso, ElasticNet

In [3]:
Hitters = load_data('Hitters')
Hitters['Salary'].isna().sum(), np.isnan(Hitters['Salary']).sum()

(59, 59)

In [4]:
design = MS(Hitters.dropna().columns.drop('Salary')).fit(Hitters.dropna())
y = np.array(Hitters.dropna()['Salary'])
X = design.transform(Hitters.dropna())
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.546
Model:                            OLS   Adj. R-squared:                  0.511
Method:                 Least Squares   F-statistic:                     15.39
Date:                Sun, 16 Feb 2025   Prob (F-statistic):           7.84e-32
Time:                        15:21:20   Log-Likelihood:                -1876.2
No. Observations:                 263   AIC:                             3792.
Df Residuals:                     243   BIC:                             3864.
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
intercept      163.1036     90.779      1.797      0.074     -15.710     341.917
AtBat           -1.9799      0.634     -3.123      0.002      -3.229      -0.731
Hits             7.5008      2.378      3.155      0.002       2.818      12.184
HmRun            4.3309      6.201      0.698      0.486      -7.885      16.546
Runs            -2.3762      2.981     -0.797      0.426      -8.248       3.495
RBI             -1.0450      2.601     -0.402      0.688      -6.168       4.078
Walks            6.2313      1.829      3.408      0.001       2.630       9.833
Years           -3.4891     12.412     -0.281      0.779     -27.938      20.960
CAtBat          -0.1713      0.135     -1.267      0.206      -0.438       0.095
CHits            0.1340      0.675      0.199      0.843      -1.195       1.463
CHmRun          -0.1729      1.617     -0.107      0.915      -3.358       3.013
CRuns            1.4543      0.750      1.938      0.054      -0.024       2.933
CRBI             0.8077      0.693      1.166      0.245      -0.557       2.172
CWalks          -0.8116      0.328     -2.474      0.014      -1.458      -0.165
League[N]       62.5994     79.261      0.790      0.430     -93.528     218.727
Division[W]   -116.8492     40.367     -2.895      0.004    -196.363     -37.335
PutOuts          0.2819      0.077      3.640      0.000       0.129       0.434
Assists          0.3711      0.221      1.678      0.095      -0.065       0.807
Errors          -3.3608      4.392     -0.765      0.445     -12.011       5.290
NewLeague[N]   -24.7623     79.003     -0.313      0.754    -180.380     130.855
==============================================================================
Omnibus:                       87.414   Durbin-Watson:                   2.018
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              452.923
Skew:                           1.236   Prob(JB):                     4.46e-99
Kurtosis:                       8.934   Cond. No.                     2.09e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.09e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [5]:
design.terms

Index(['AtBat', 'Hits', 'HmRun', 'Runs', 'RBI', 'Walks', 'Years', 'CAtBat',
       'CHits', 'CHmRun', 'CRuns', 'CRBI', 'CWalks', 'League', 'Division',
       'PutOuts', 'Assists', 'Errors', 'NewLeague'],
      dtype='object')

In [6]:
strategy = Stepwise.fixed_steps(design, len(design.terms), direction='forward')
full_path = sklearn_selection_path(sm.OLS, strategy)

In [7]:
full_path.fit(Hitters.dropna(), y)
yhat_in = full_path.predict(Hitters.dropna())
yhat_in.shape

c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_d

(263, 20)

In [8]:
yhat_in

array([[ 535.92588213,  602.03516506,  505.03086087, ...,  372.19029353,
         361.43503052,  362.13606585],
       [ 535.92588213,  484.97403646,  564.66713304, ...,  706.17406201,
         712.13809677,  712.6952059 ],
       [ 535.92588213,  937.39947944,  995.59888206, ..., 1166.41289971,
        1171.29048328, 1171.31111145],
       ...,
       [ 535.92588213,  348.13906857,  432.11115796, ...,  247.09365751,
         245.71268797,  243.69097639],
       [ 535.92588213,  606.78088649,  717.12346513, ..., 1026.13008776,
        1023.92340166, 1023.30029814],
       [ 535.92588213,  556.95081148,  759.48176151, ...,  708.6674326 ,
         703.9112215 ,  705.7780666 ]])

In [9]:
full_path

sklearn_selection_path(model_type=<class 'statsmodels.regression.linear_model.OLS'>,
                       strategy=Strategy(initial_state=(), candidate_states=<bound method Stepwise.candidate_states of <ISLP.models.strategy.Stepwise object at 0x0000015F2412BC70>>, build_submodel=<bound method ModelSpec.build_submodel of ModelSpec(terms=Index(['AtBat', 'Hits', 'HmRun', 'Runs', 'RBI', 'Walks', 'Years', 'CAtBat',
       'CHits', 'CHmRun', 'CRuns', 'CRBI', 'CWalks', 'League', 'Division',
       'PutOuts', 'Assists', 'Errors', 'NewLeague'],
      dtype='object'))>, check_finished=functools.partial(<function fixed_steps at 0x0000015F26C87880>, 19), postprocess=functools.partial(<function _postprocess_fixed_steps at 0x0000015F26C879A0>, 19)))

In [10]:
cross_val_predict?

Signature:
cross_val_predict(
    estimator,
    X,
    y=None,
    *,
    groups=None,
    cv=None,
    n_jobs=None,
    verbose=0,
    fit_params=None,
    pre_dispatch='2*n_jobs',
    method='predict',
)
Docstring:
Generate cross-validated estimates for each input data point.

The data is split according to the cv parameter. Each sample belongs
to exactly one test set, and its prediction is computed with an
estimator fitted on the corresponding training set.

Passing these predictions into an evaluation metric may not be a valid
way to measure generalization performance. Results can differ from
:func:`cross_validate` and :func:`cross_val_score` unless all tests sets
have equal size and the metric decomposes over samples.

Read more in the :ref:`User Guide <cross_validation>`.

Parameters
----------
estimator : estimator object implementing 'fit' and 'predict'
    The object to use to fit the data.

X : array-like of shape (n_samples, n_features)
    The data to fit. Can be, for exam

In [11]:
yhat_cv = cross_val_predict(full_path, Hitters.dropna(), y, cv=5)
yhat_cv

c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_d

array([[ 556.94379524,  621.85905125,  517.41674257, ...,  289.06675386,
         290.91123158,  291.32035967],
       [ 556.94379524,  501.11900596,  572.42673549, ...,  734.05275992,
         735.77237915,  735.98970301],
       [ 556.94379524,  967.76296479, 1032.3521729 , ..., 1237.90254622,
        1241.47819013, 1241.95654469],
       ...,
       [ 545.00871564,  343.10453875,  456.39482611, ...,  199.74364217,
         194.16553392,  194.20692307],
       [ 545.00871564,  602.62953023,  738.53571019, ..., 1177.24176005,
        1173.86475543, 1173.92719816],
       [ 545.00871564,  552.62930252,  794.69345996, ...,  646.12957458,
         666.69764742,  666.81188246]])

In [12]:
yhat_cv.shape

(263, 20)

In [23]:
Hitters = Hitters.dropna()
y = Hitters['Salary'].values
X = Hitters.drop('Salary', axis=1)
X = X.select_dtypes(include=np.number)
X, y

(     AtBat  Hits  HmRun  Runs  RBI  Walks  Years  CAtBat  CHits  CHmRun  \
 1      315    81      7    24   38     39     14    3449    835      69   
 2      479   130     18    66   72     76      3    1624    457      63   
 3      496   141     20    65   78     37     11    5628   1575     225   
 4      321    87     10    39   42     30      2     396    101      12   
 5      594   169      4    74   51     35     11    4408   1133      19   
 ..     ...   ...    ...   ...  ...    ...    ...     ...    ...     ...   
 317    497   127      7    65   48     37      5    2703    806      32   
 318    492   136      5    76   50     94     12    5511   1511      39   
 319    475   126      3    61   43     52      6    1700    433       7   
 320    573   144      9    85   60     78      8    3198    857      97   
 321    631   170      9    77   44     31     11    4908   1457      30   
 
      CRuns  CRBI  CWalks  PutOuts  Assists  Errors  
 1      321   414     375      6

In [34]:
X_scale = (X - X.mean(axis=0)) / X.std(0)
lambdas = 10 ** np.linspace(8, -2, 100) / y.std()
soln_array = ElasticNet.path(X_scale, y, l1_ratio=0, alphas=lambdas)
soln_array[1].shape

c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:617: UserWarning: Coordinate descent without L1 regularization may lead to unexpected results and is discouraged. Set l1_ra

(16, 100)

In [35]:
soln_array[1]

array([[ 7.98787629e-04,  1.00794729e-03,  1.27187188e-03, ...,
        -3.24279110e+02, -3.24328635e+02, -3.24368125e+02],
       [ 8.87626606e-04,  1.12004947e-03,  1.41332896e-03, ...,
         3.52518444e+02,  3.52664253e+02,  3.52780499e+02],
       [ 6.94087571e-04,  8.75831079e-04,  1.10516058e-03, ...,
         1.85777691e+01,  1.86534743e+01,  1.87137598e+01],
       ...,
       [ 6.08006168e-04,  7.67213051e-04,  9.68107150e-04, ...,
         8.24558391e+01,  8.24569404e+01,  8.24578239e+01],
       [ 5.14685717e-05,  6.49456209e-05,  8.19515205e-05, ...,
         5.56525366e+01,  5.56646361e+01,  5.56741813e+01],
       [-1.09287990e-05, -1.37907659e-05, -1.74022685e-05, ...,
        -1.90600809e+01, -1.90515677e+01, -1.90447745e+01]])

In [41]:
ridge = ElasticNet(l1_ratio=0)
scaler = StandardScaler()
pipe = make_pipeline(scaler, ridge)
param_grid = {'elasticnet__alpha':np.logspace(-2, 3, 6)}
grid = GridSearchCV(pipe, param_grid=param_grid, cv=5, n_jobs=-1)
grid.fit(X, y)

c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\admin\anaconda3\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  i

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('elasticnet', ElasticNet(l1_ratio=0))]),
             n_jobs=-1,
             param_grid={'elasticnet__alpha': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03])})

In [44]:
grid.best_estimator_.named_steps['elasticnet'].coef_

array([13.79910227, 34.83204351,  6.44492558, 26.39987956, 23.14755202,
       33.19306807,  9.70146617, 25.66237694, 34.17484501, 30.33762148,
       35.16404782, 35.50661701, 17.66451249, 41.54952635,  3.60515041,
       -6.59434624])